In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from implicit.datasets.movielens import get_movielens
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import tqdm
import codecs

In [2]:
retail_data = pd.read_json('data/cus_user_quality_date_0716.json') # This may take a couple minutes
# df2 = pd.read_csv('data/product_brand_type_color.csv') # This may take a couple minutes

In [3]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120442 entries, 0 to 120441
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   120442 non-null  int64  
 1   PRODUCT_DIM_KEY    120442 non-null  int64  
 2   QUANTITY           120441 non-null  float64
 3   PRODUCT_NAME       120435 non-null  object 
 4   COLOR              120442 non-null  object 
 5   PRODUCT_TYPE       120442 non-null  object 
 6   CATEGORY           120442 non-null  object 
 7   BRAND              120442 non-null  object 
 8   RESOLUTION_STATUS  120442 non-null  object 
 9   ORDER_DATE_KEY     120442 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


In [176]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail['RESOLUTION_STATUS'] == 'COMPLETED']
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

Duplicated rows: 272


In [177]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48715 entries, 0 to 120395
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CUSTOMER_DIM_KEY   48715 non-null  int64  
 1   PRODUCT_DIM_KEY    48715 non-null  int64  
 2   QUANTITY           48715 non-null  float64
 3   PRODUCT_NAME       48715 non-null  object 
 4   COLOR              48715 non-null  object 
 5   PRODUCT_TYPE       48715 non-null  object 
 6   CATEGORY           48715 non-null  object 
 7   BRAND              48715 non-null  object 
 8   RESOLUTION_STATUS  48715 non-null  object 
 9   ORDER_DATE_KEY     48715 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.1+ MB


In [178]:
item_lookup = cleaned_retail[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['PRODUCT_DIM_KEY'] = item_lookup.PRODUCT_DIM_KEY.astype(str) # Encode as strings for future lookup ease

In [7]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PRODUCT_DIM_KEY'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [197]:
df_lim = threshold_likes(cleaned_retail, 1, 2)

Starting likes info
Number of users: 25852
Number of models: 16300
Sparsity: 0.012%
Ending likes info
Number of users: 8753
Number of models: 12843
Sparsity: 0.028%


In [198]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(grouped_purchased[col_feature].unique()) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases

    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse

In [199]:
cleaned_retail = df_lim
cleaned_retail['CUSTOMER_DIM_KEY'] = cleaned_retail['CUSTOMER_DIM_KEY'].astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['PRODUCT_DIM_KEY', 'QUANTITY', 'CUSTOMER_DIM_KEY']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY']).sum().reset_index() # Group together
grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive

customers = list(np.sort(grouped_purchased['CUSTOMER_DIM_KEY'].unique())) # Get our unique customers
products = list(grouped_purchased['PRODUCT_DIM_KEY'].unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.QUANTITY) # All of our purchases

rows = grouped_purchased.CUSTOMER_DIM_KEY.astype('category', CategoricalDtype(categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased.PRODUCT_DIM_KEY.astype('category', CategoricalDtype(categories = products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

G:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [200]:
#purchases_sparse = create_matrix(df_lim, 'CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY')
purchases_sparse = create_matrix(df_lim, 'PRODUCT_DIM_KEY', 'CUSTOMER_DIM_KEY')

In [12]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [238]:
#product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)

In [201]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [205]:
model = AlternatingLeastSquares()
ratings = (bm25_weight(product_train, B=0.9) * 5).tocsr()
model.fit(ratings*15)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.667, 0.682)

In [189]:
len(model.item_factors)

7590

In [15]:
import time
def train_model(ratings, model_name, factors =20, regularization=0.1,alpha = 10):
    # lets weight these models by bm25weight.

    if model_name == "als":
        model = AlternatingLeastSquares(factors =factors, regularization=regularization)
        print("weighting matrix by bm25_weight")
        ratings = (bm25_weight(ratings, B=0.9) * 5).tocsr()
    elif model_name == "bpr":
        model = BayesianPersonalizedRanking()

    elif model_name == "lmf":
        model = LogisticMatrixFactorization()

    elif model_name == "tfidf":
        model = TFIDFRecommender()

    elif model_name == "cosine":
        model = CosineRecommender()

    elif model_name == "bm25":
        model = BM25Recommender(B=0.2)

    else:
        raise NotImplementedError("TODO: model %s" % model_name)
    # train the model
    print("training model %s", model_name)
    start = time.time()
    model.fit(ratings*alpha)
    print("trained model '%s' in %s", model_name, time.time() - start)
        
    item_vecs = model.user_factors
    user_vecs = model.item_factors
    return item_vecs, user_vecs, model

In [16]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [17]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [18]:
def calc_mean_auc_cv(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [19]:
def calc_mean_auc_pop(training_set, altered_users, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    
    return float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [70]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_als(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9) * 5).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)

    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
                    best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best factor: ', best_factors)
    print('Best regularization: ', best_reg)
    print('Best alpha: ', best_alpha)
    return best_auc, pop_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [42]:
cross_validation_als(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.725



10 0.0 1 0.801



10 0.0 10 0.82



10 0.0 50 0.809



10 0.0 100 0.802


...


120 100.0 50 0.809



120 100.0 100 0.797



120 100.0 500 0.791



120 100.0 1000 0.78
Best AUC:  0.83
Pop AUC:  0.725
Best factor:  80
Best regularization:  100.0
Best alpha:  10


(0.83, 0.725, 80, 100.0, 10)

In [31]:
cross_validation_als(purchases_sparse, 'als', param_grid)


10 0.0 1 0.805 0.742



10 0.0 10 0.823 0.742



10 0.0 50 0.825 0.742



10 0.0 100 0.818 0.742



10 0.0 500 0.798 0.742


...


120 100.0 50 0.815 0.742



120 100.0 100 0.801 0.742



120 100.0 500 0.791 0.742



120 100.0 1000 0.789 0.742
0.84 40 100.0 10


(0.84, 40, 100.0, 10)

In [ ]:
# bpr lmf als bm25

item_vecs, user_vecs, model = train_model(product_train, 'lmf')
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

In [54]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001]
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [0.1],
              'regularization': [0.1],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_bpr(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = BayesianPersonalizedRanking(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [55]:
cross_validation_bpr(purchases_sparse, 'bpr', param_grid)


10 0.1 0.1 1 0.822 0.742



10 0.1 0.1 10 0.825 0.742



10 0.1 0.1 50 0.82 0.742



10 0.1 0.1 100 0.824 0.742



10 0.1 0.1 500 0.826 0.742



10 0.1 0.1 1000 0.818 0.742


KeyboardInterrupt: 

In [56]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001],
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001],
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation_lmf(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = LogisticMatrixFactorization(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [57]:
cross_validation_lmf(purchases_sparse, 'lmf', param_grid)


10 0.0 1 1 0.772 0.742


KeyboardInterrupt: 

In [140]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [24]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items  
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    #print(purchased_ind)
    return item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)]


In [141]:
len(customers_arr)

6684

In [80]:
item_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16300 entries, 0 to 120395
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_DIM_KEY  16300 non-null  int64 
 1   PRODUCT_NAME     16300 non-null  object
dtypes: int64(1), object(1)
memory usage: 382.0+ KB


In [84]:
customers_arr[:30]

array([31785, 31786, 31795, 31807, 31821, 31835, 31842, 31844, 31856,
       31868, 31902, 31926, 31960, 32005, 32008, 32027, 32030, 32034,
       32053, 32054, 32060, 32066, 32096, 32099, 32104, 32133, 32138,
       32166, 32173, 32191], dtype=int64)

In [145]:
get_items_purchased(31785, product_test, customers_arr, products_arr, item_lookup)

IndexError: index 5370 is out of bounds for axis 0 with size 4980

In [146]:
rec_items(31785, product_test, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 20)

IndexError: index 5548 is out of bounds for axis 0 with size 4980

In [92]:
len(products_arr)

4980

In [150]:
def get_rec_items(model, customer_id, sparse_matrix, customer_list, item_list, item_lookup, num_items = 10):
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = sparse_matrix[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = item_list[purchased_ind] # Get the stock codes for our purchased items
    
    print(item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)])
    
    recom = model.recommend(cust_ind, sparse_matrix, N= num_items)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in recom:
        print(index)
        key = item_list[index[0]]
        rec_keys.append(key)
        descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in recom]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': rec_keys, 'PRODUCT_NAME': descriptions, 'SCORE': scores}) # Create a dataframe 
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']] # Switch order of columns around

get_rec_items(model, 31795, product_train, customers_arr, products_arr, item_lookup, num_items = 20)

       PRODUCT_DIM_KEY                          PRODUCT_NAME
88735           517345  Washed Linen Sky Quilt Sham European
(3639, 1.0008551)
(3638, 1.000838)
(1562, 0.9990205)
(313, 0.99773204)
(1563, 0.9958519)
(1181, 0.99521506)
(317, 0.99337155)
(2847, 0.99314594)
(3564, 0.9915873)
(2300, 0.99107176)
(2838, 0.98900735)
(312, 0.889523)
(2865, 0.85162026)
(377, 0.84371364)
(3567, 0.833702)
(3561, 0.7893115)
(3569, 0.78742146)
(2817, 0.70117974)
(37, 0.687875)
(3647, 0.6817236)


,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,478177,Dannes Light Gray Soap Dish Rectangular Straig...,1.000855
1,347675,Manchester Sand Wastebasket Square Tapered Sha...,1.000838
2,505671,Blaine - Oblong Tablecloth 70x108,0.999021
3,142108,Place Cards (Box of 25),0.997732
4,499089,Vecchio Ginori 6 1/4 In. Fruit Bowl Vecchio Gi...,0.995852
5,176275,Belmont Dove White Mug 12 oz,0.995215
6,405441,"Kensington Lions, Pair",0.993372
7,313961,Titanic Tea Cup,0.993146
8,498514,Antiqua Turquoise Melamine 6.5â?? Appetizer Pl...,0.991587
9,333835,Rolling Stone Acrylic Flute Turquoise,0.991072


In [32]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': codes, 'PRODUCT_NAME': descriptions}) # Create a dataframe 
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']] # Switch order of columns around

In [174]:
model.similar_items(90,N=20)

[(90, 0.048315637),
 (94, 0.047888927),
 (91, 0.047666892),
 (93, 0.047576323),
 (89, 0.047496896),
 (88, 0.047284957),
 (87, 0.046977323),
 (92, 0.046765365),
 (95, 0.046393476),
 (96, 0.045296676),
 (97, 0.044456724),
 (1362, 0.041736342),
 (1329, 0.041545182),
 (4252, 0.039946917),
 (755, 0.039921522),
 (3809, 0.039634597),
 (4015, 0.03957007),
 (4537, 0.03956089),
 (2683, 0.039501112),
 (3807, 0.03944798)]

In [168]:
def get_similar_items(model, item_id, customer_list, item_list, item_lookup, num_items = 10):
    item_ind = np.where(item_list == item_id)[0][0] # Returns the index row of our item id
    sim_list = [] # start empty list to store items
    for index, score in model.similar_items(item_ind, N=num_items):
        code = item_list[index]
        sim_list.append([code, item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == code].iloc[0]]) 
    codes = [item[0] for item in sim_list]
    descriptions = [item[1] for item in sim_list]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': codes, 'PRODUCT_NAME': descriptions}) # Create a dataframe 
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']] # Switch order of columns around
get_similar_items(model, 194662, customers_arr, products_arr, item_lookup, num_items = 20)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,194662,Giza 45 Percale Standard Pillowcases (22x33 In...
1,299942,Cassandre Grenat Napkin 21 X 21 100% Cotton
2,172750,Fortuny Dessert Plates (Mixed) set of 4
3,205390,Hammam Bath Towel 30 x 54 in
4,194304,Juliet Four Napkins 22 X 22 In
5,194286,Juliet 106 In Round Tablecloth
6,105698,Festival Tablecloth 66 X 106 in Rect
7,196528,Living Asparagus tong 9 in.
8,299944,"Cassandre Grenat Placemat 21 X 15, 100% Cotton..."
9,347298,Andria Aqua Brush Holder Square Tapered


In [167]:
products_arr[90]

194662

In [173]:
item_ind = np.where(products_arr == 299942)[0][0] # Returns the index row of our item id
item_ind

94